# API Data Pipeline

### MN GIS API - Infested Waters Shapefile

In [ ]:
# Import necessary packages
import requests
import json
import zipfile
import io
import pandas as pd
import os

# Set access url
url="https://gisdata.mn.gov/api/3/action/package_show?id=c4be2e23-2c68-47ab-8b12-1b1e6df264fb"

# Pull necessary data
r = requests.get(url, verify=False)
print('Data pulled.')

In [ ]:
# Convert the data into json, and then into a dataframe and print it
json = r.json()
df = pd.DataFrame.from_dict(json)
df

In [ ]:
# Get details for resources and pull out the shapefile's URL
resources = df.loc['resources']
reso_resu = pd.DataFrame.from_dict(resources['result'])
shape_url = reso_resu.loc[reso_resu['resource_type'] == 'shp']['url'].tolist()
print("Shapefile URL: ", shape_url[0])

In [ ]:
# Download the zip file at the URL obtained
file_req = requests.get(shape_url[0])
z = zipfile.ZipFile(io.BytesIO(file_req.content))
z.extractall()

### Google API - Nearby Search

In [ ]:
# Find Place Google API
import requests
import pandas as pd
import geopandas as gpd
import json

url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=44.986656%2C-93.258133&radius=1500&type=establishment&keyword=water&key=YOURKEY"

payload= {}
headers = {}

# Use set url to get JSON data
response = requests.request("GET", url, headers=headers, data=payload)
text_dat = response.text
results = response.json()["results"]

In [ ]:
# Create normalized data frame
df = pd.json_normalize(results)
df

In [ ]:
# Rename coordinate columns with more useful titles
df.rename(columns = {'geometry.location.lat':'Latitude'}, inplace = True)
df.rename(columns = {'geometry.location.lng':'Longitude'}, inplace = True)

# Create geodataframe using the dataframe and renamed coordinate columns, set crs
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Latitude, df.Longitude), crs = 26915)

# Show the new geodataframe
print(gdf.head)

### Rectify Coordinate Systems

In [ ]:
# Read shapefile data
infested_waters = gpd.read_file("/home/sauer326/5571 Labs/env_listed_infested_waters.shp")

# View the coordinate reference system of both layers 
print(infested_waters.crs)
print(gdf.crs)

In [ ]:
# Plot vector data to confirm its validity
infested_waters.plot()
gdf.plot()

### Perform Spatial Join

In [ ]:
# Use Geopandas spatial join to combine two datasets
joined_data = gpd.sjoin(infested_waters, gdf)

### References
https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.to_file.html

https://shapely.readthedocs.io/en/stable/manual.html

https://www.geeksforgeeks.org/json-loads-in-python/#:~:text=loads()%20method%20can%20be,JSON%20data%20into%20Python%20Dictionary.

https://sparkbyexamples.com/pandas/pandas-read-json-with-examples/#:~:text=pandas%20read_json()%20function%20can,two%20systems%20or%20web%20applications

https://stackoverflow.com/questions/64390923/extract-location-coordinates-from-json-results

https://www.earthdatascience.org/workshops/gis-open-source-python/reproject-vector-data-in-python/

https://stackoverflow.com/questions/41168558/python-how-to-convert-json-file-to-dataframe

https://geopandas.org/en/stable/gallery/create_geopandas_from_pandas.html

https://geopandas.org/en/stable/gallery/spatial_joins.html

https://docs.ckan.org/en/ckan-2.1.5/api.html

https://automating-gis-processes.github.io/CSC18/lessons/L4/spatial-join.html